In [7]:
import re

# Modified function to handle multiple institutions
def create_training_data_multiple_affiliations(templates, institutions):
    training_data = []
    
    for template in templates:
        text = template
        entities = []
        
        # For each institution, find its position in the text
        for institution in institutions:
            entity_start = text.find(institution)
            if entity_start != -1:  # Only include if found
                entity_end = entity_start + len(institution)
                entities.append((entity_start, entity_end, "INSTITUTION"))
        
        # Append the text and entity annotations if at least one institution was found
        if entities:
            training_data.append((text, {"entities": entities}))
    
    return training_data

# Example usage
templates = [
    """\\author{
    Danielle C. Maddix \\\\ Amazon Research \\\\ 2795 Augustine Dr. \\\\ Santa Clara, CA 95054 \\\\ \texttt{dmmaddix@amazon.com} \\\\
    \And
    Nadim Saad \\\\ Stanford University \\\\ 450 Serra Mall \\\\ Stanford, CA 94305 \\\\ \texttt{nsaad31@stanford.edu} \\\\
    \And
    Yuyang Wang \\\\ Amazon Research \\\\ 2795 Augustine Dr. \\\\ Santa Clara, CA 95054 \\\\ \texttt{yuyawang@amazon.com} \\\\
    }"""
]

institutions = ["Amazon Research", "Stanford University"]

# Generate training data
training_data = create_training_data_multiple_affiliations(templates, institutions)
print(training_data)

import nlp
# Save the trained model to disk
output_dir = "./trained_ner_model"
nlp.to_disk(output_dir)
print(f"Model saved to {output_dir}")


[('\\author{\n    Danielle C. Maddix \\\\ Amazon Research \\\\ 2795 Augustine Dr. \\\\ Santa Clara, CA 95054 \\\\ \texttt{dmmaddix@amazon.com} \\\\\n    \\And\n    Nadim Saad \\\\ Stanford University \\\\ 450 Serra Mall \\\\ Stanford, CA 94305 \\\\ \texttt{nsaad31@stanford.edu} \\\\\n    \\And\n    Yuyang Wang \\\\ Amazon Research \\\\ 2795 Augustine Dr. \\\\ Santa Clara, CA 95054 \\\\ \texttt{yuyawang@amazon.com} \\\\\n    }', {'entities': [(35, 50, 'INSTITUTION'), (159, 178, 'INSTITUTION')]})]


2024-10-20 22:07:14.794917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: module 'nlp' has no attribute 'to_disk'

In [7]:
from fuzzywuzzy import process

def match_to_ror(extracted_affiliation, ror_institutions):
    best_match, score = process.extractOne(extracted_affiliation, ror_institutions)
    return best_match if score > 80 else None


In [8]:
import json

# Function to extract institution names and aliases from ROR data
def extract_institution_names(ror_data):
    institution_names = []
    for entry in ror_data:
        for name_entry in entry['names']:
            institution_names.append(name_entry['value'])
    return institution_names

# Load the ROR dataset from JSON
with open("ror.json", "r") as f:
    ror_data = json.load(f)

# Extract all institution names
institution_names = extract_institution_names(ror_data)


In [5]:
import random

def create_training_data(institution_names, templates):

    
    training_data = []
    for institution in institution_names:
        template = random.choice(templates)
        text = template.format(institution)
        entity_start = text.find(institution)
        entity_end = entity_start + len(institution)
        
        training_data.append((text, {"entities": [(entity_start, entity_end, "INSTITUTION")]}))
    
    return training_data

# Create training data using the ROR institution names
TRAIN_DATA = create_training_data(institution_names)


TypeError: create_training_data() missing 1 required positional argument: 'templates'

In [ ]:
import re
import json
from fuzzywuzzy import process

# Load the ROR JSON database
with open("ror.json", "r") as f:
    ror_data = json.load(f)

# Extract institution names from ROR data
def extract_ror_institutions(ror_data):
    institution_names = []
    for entry in ror_data:
        for name in entry['names']:
            institution_names.append(name['value'])
    return institution_names

ror_institution_names = extract_ror_institutions(ror_data)

# Function to extract affiliations from LaTeX
def extract_affiliations_from_latex(latex_text):
    affiliation_pattern = r'\\affiliation\{(.*?)\}'
    affiliations = re.findall(affiliation_pattern, latex_text, re.DOTALL)
    cleaned_affiliations = [affil.replace('\n', ' ').strip() for affil in affiliations]
    return cleaned_affiliations

# Match extracted affiliation to ROR institution names using fuzzy matching
def match_to_ror(extracted_affiliation, ror_institutions):
    best_match, score = process.extractOne(extracted_affiliation, ror_institutions)
    return best_match if score > 80 else None

# Generate training data using LaTeX and ROR names
def create_training_data_from_latex_ror(latex_text, ror_institution_names):
    affiliations = extract_affiliations_from_latex(latex_text)
    training_data = []
    
    for affiliation in affiliations:
        matched_ror = match_to_ror(affiliation, ror_institution_names)
        if matched_ror:
            entity_start = latex_text.find(affiliation)
            entity_end = entity_start + len(affiliation)
            training_data.append((latex_text, {"entities": [(entity_start, entity_end, "INSTITUTION")]}))
    
    return training_data

# Example LaTeX text
latex_text = r'''
\author{Giorgio Busoni}
 \email{giorgio.busoni@mpi-hd.mpg.de}
\affiliation{
 Max-Planck-Institut fur Kernphysik,\\ Saupfercheckweg 1, 69117 Heidelberg, Germany
}
'''

# Generate training data
training_data = create_training_data_from_latex_ror(latex_text, ror_institution_names)
print("Generated Training Data:", training_data)


In [2]:
# 10.20
import json
import random
import spacy
from spacy.training import Example

# Load ROR dataset
with open('ror.json', 'r') as f:
    ror_data = json.load(f)

# Extract institution names
def extract_institution_names(ror_data):
    institution_names = []
    for entry in ror_data:
        for name_entry in entry['names']:
            institution_names.append(name_entry['value'])
    return institution_names

institution_names = extract_institution_names(ror_data)

# Create training examples from the ROR institution names
def create_training_data_from_ror(institution_names, num_samples=1000):
    templates = [
        "{} is a leading research institute.",
        "The work was done at {}.",
        "The collaboration between {} and other universities is remarkable.",
        "{} has been a pioneer in the field."
    ]
    
    training_data = []
    for _ in range(num_samples):
        institution = random.choice(institution_names)
        template = random.choice(templates)
        text = template.format(institution)
        entity_start = text.find(institution)
        entity_end = entity_start + len(institution)
        training_data.append((text, {"entities": [(entity_start, entity_end, "INSTITUTION")]}))
    
    return training_data

# Create training data
TRAIN_DATA = create_training_data_from_ror(institution_names)


In [9]:
# Create blank spaCy model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Add the "INSTITUTION" label
ner.add_label("INSTITUTION")

# Prepare the training data for spaCy
def train_ner_model(train_data):
    optimizer = nlp.begin_training()

    for itn in range(10):  # 10 iterations of training
        random.shuffle(train_data)
        losses = {}
        for text, annotations in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, losses=losses)
        print(f"Iteration {itn} - Losses: {losses}")

# Train the NER model
train_ner_model(TRAIN_DATA)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The work was done at Centurion Ilac San. ve Tic A...." with entities "[(21, 52, 'INSTITUTION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The work was done at Clinical Tools, Inc.." with entities "[(21, 41, 'INSTITUTION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/training/iob_utils.py

Iteration 0 - Losses: {'ner': 1037.5942916136526}
Iteration 1 - Losses: {'ner': 160.33088823126627}


KeyboardInterrupt: 

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into train and test sets
train_data, test_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)





# Function to test the model on unseen data
def test_ner_model(test_data, model):
    correct_predictions = 0
    total_entities = 0
    all_entities = []

    for text, annotations in test_data:
        doc = model(text)
        predicted_entities = [(ent.text, ent.label_) for ent in doc.ents]
        true_entities = annotations['entities']

        # Check if the predicted entities match the true entities
        for pred_entity in predicted_entities:
            if any(text[start:end] == pred_entity[0] for start, end, label in true_entities):
                correct_predictions += 1
        
        total_entities += len(true_entities)
        all_entities.append((predicted_entities, true_entities))

    accuracy = correct_predictions / total_entities if total_entities else 0
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return all_entities

# # Evaluate the model
# test_ner_model(test_data, nlp)

test_paragraph = """
The collaboration between Amazon Research and Stanford University resulted in significant advancements.
"""

doc = nlp(test_paragraph)
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")


Entity: Amazon Research and Stanford University resulted in significant advancements, Label: INSTITUTION


In [5]:
ner = nlp.get_pipe("ner")
def fine_tune_ner_model(train_data):
    # Add the "ORG" label if not already present
    ner.add_label("ORG")
    
    # Disable other pipelines during training, except 'ner'
    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

    # Begin training
    with nlp.disable_pipes(*unaffected_pipes):  # only train NER
        optimizer = nlp.resume_training()

        for itn in range(10):  # 10 training iterations
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.5, losses=losses)
            print(f"Iteration {itn}, Losses: {losses}")

# Fine-tune the pre-trained model
fine_tune_ner_model(TRAIN_DATA)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The work was done at Clinical Tools, Inc.." with entities "[(21, 41, 'INSTITUTION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The work was done at IfM." with entities "[(21, 24, 'INSTITUTION')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] So

Iteration 0, Losses: {'ner': 10.966555260701712}
Iteration 1, Losses: {'ner': 27.075484868097707}
Iteration 2, Losses: {'ner': 15.901708882065583}
Iteration 3, Losses: {'ner': 27.467168395733385}
Iteration 4, Losses: {'ner': 13.9366148383876}
Iteration 5, Losses: {'ner': 5.31615356069249}
Iteration 6, Losses: {'ner': 30.795950481227877}
Iteration 7, Losses: {'ner': 9.84563931631919}
Iteration 8, Losses: {'ner': 10.236311512922503}
Iteration 9, Losses: {'ner': 9.498716846197235}


In [6]:
# Save the model
nlp.to_disk("./fine_tuned_ner_model")
print("Model saved!")

# Load the model for testing
nlp_fine_tuned = spacy.load("./fine_tuned_ner_model")

# Test on a paragraph
test_paragraph = """
The collaboration between Amazon Research and Stanford University resulted in significant advancements.
"""

doc = nlp_fine_tuned(test_paragraph)
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")


Model saved!
Entity: Amazon Research and Stanford University, Label: INSTITUTION
Entity: .
, Label: INSTITUTION


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into training and test sets
train_data, test_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)

# Evaluate on the test data
correct_predictions = 0
total_predictions = 0

for text, annotations in test_data:
    doc = nlp_fine_tuned(text)
    predicted_entities = [(ent.text, ent.label_) for ent in doc.ents]
    true_entities = [(text[start:end], label) for start, end, label in annotations["entities"]]
    
    for pred_entity in predicted_entities:
        if pred_entity in true_entities:
            correct_predictions += 1
    
    total_predictions += len(true_entities)

accuracy = correct_predictions / total_predictions if total_predictions else 0
print(f"Accuracy: {accuracy * 100:.2f}%")
